## Conduct the CNN model training based on the im2recipe image dataset and the topic modeling(NMF)

## Import 1.Module  2.Layer1&Layer2 json  3.Create title cleaned validation & test dataset

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import os, csv
import json  
import pandas as pd
import numpy as np
import re
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim.corpora as corpora
from operator import itemgetter
import operator
import pickle
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms
import cv2
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader 
import time
import copy
import torch.nn.functional as F
os.chdir("/content/gdrive/My Drive/2021 Summer CS7643 Share Folder/data/recipe1M")
layer1 = json.load(open("layer1.json", "r"))
layer2 = json.load(open("layer2.json", "r"))

Mounted at /content/gdrive
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# create title cleaned dataframe for val/test dataset, please enter true or false
create_val_df = True
create_test_df = True
create_train_df = False

# No need to enter, just to store dataframe in case we need it for visualization
df_title_val = None # dataframe with cleaned title for validation dataset
df_title_test = None # dataframe with cleaned title for test dataset
df_title_traing = None # dataframe with cleaned title for test dataset


# ======== No need to modify code below ============================================
# function to create the cleaned title dataframe work
def create_clean_titled_df(data_type): # datatype should be either "val" or "test"
  layer1_data_list = []
  for i in range(len(layer1)):
    temp = {}
    if layer1[i]["partition"] == data_type:
      temp["id"] = layer1[i]["id"]
      temp['title'] = layer1[i]['title']
      layer1_data_list.append(temp)
  df_title = pd.DataFrame(layer1_data_list)
  # Remove everything that is not alphanumeric or underscore
  df_title['title_clean'] = df_title['title'].apply(lambda x: re.sub(r'[^\w]', ' ', x))
  # Convert the titles to lowercase
  df_title['title_clean'] = df_title['title_clean'].apply(lambda x: x.lower())
  return df_title

if create_val_df:
  df_title_val = create_clean_titled_df("val")
  df_title_val.to_csv("layer1_title_val.csv")
if create_test_df:
  df_title_test = create_clean_titled_df("test")
  df_title_test.to_csv("layer1_title_test.csv")
if create_train_df:
  df_title_train = create_clean_titled_df("train")
  df_title_train.to_csv("layer1_title_train.csv")

## Apply topic modeling on val/test dataset for categorization

In [3]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word.strip() for word in simple_preprocess(str(doc)) 
             if word.strip() not in stop_words] for doc in texts]
def get_top_cat_for_recipe(recipes_all_list, w_vector):
    final_recipes_cat_name = []
    recipe_cat_link = []
    for r in range(len(recipes_all_list)):
        one_hot_matrix = w_vector[r,:]
        all_zeros = not one_hot_matrix.any() # no fitting category
        if all_zeros: recipe_cat_link.append(-1) # no fitting category
        else:
            top_indic = np.argsort(one_hot_matrix)[::-1][0] # 1. umdrehen 2. return erste
            recipe_cat_link.append(top_indic)
    for num in recipe_cat_link:
        if num == -1: final_recipes_cat_name.append('no_cat')
        else: final_recipes_cat_name.append(topics_category_300[num])
    return final_recipes_cat_name
def topic_ext (list_dic):
  new_list = []
  for i in list_dic:
    if i == "no_cat":
      new_list.append(i)
    else:
      new_list.append(i['topic'])
  return new_list

In [4]:
# build the nmf model for topic categorization
training_data = "val" # argument: "val", "test", "train"
testing_data = "test" # argument: "val", "test", "train"
training_df = None # to be assigned based on the training_data argument
testing_df = None # to be assigned based on the testing_data argument

n_topics = 300 # number of topics
nmf_model = None # built NMF model, no need to enter

nmf_training_set_topic = None # topic for each row in training set
nmf_testing_set_topic = None # topic for each row in testing set

# ======== No need to modify code below ======================================================
# get the corresponding topic for trainig and testing dataset
if training_data == "val":
  training_df = df_title_val
elif training_data == "train":
  training_df = df_title_traing
elif training_data == "test":
  training_df = df_title_test
if testing_data == "val":
  testing_df = df_title_val
elif testing_data == "train":
  testing_df = df_title_traing
elif testing_data == "test":
  testing_df = df_title_test

# creating files for training and validation
df_title_val = pd.read_csv("layer1_title_val.csv")
df_title_test = pd.read_csv("layer1_title_test.csv")
stop_words = stopwords.words('english')
stop_words.extend(['from', 'with', 'and', 'recipe', 'in', 'a', 's'])
data = training_df["title_clean"].values.tolist()
data_words = list(sent_to_words(data))
data_words = remove_stopwords(data_words)
data_test = testing_df["title_clean"].values.tolist()
data_words_test = list(sent_to_words(data_test))
data_words_test = remove_stopwords(data_words_test)
# building nmf model
data_samples = [' '.join(item) for item in data_words]
tfidf_vectorizer = TfidfVectorizer(max_df=0.99, max_features=None)
tfidf = tfidf_vectorizer.fit_transform(data_samples) #tfidf.shape #(144128, 21930)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
pickle.dump(tfidf_vectorizer, open("vectorizer_nmf_07262021.pickle", "wb")) #Save vectorizer
nmf_300 = NMF(n_components=n_topics, random_state=1, alpha=.1, l1_ratio=.5, verbose=2, max_iter=5).fit(tfidf)
nmf_embedding = nmf_300.transform(tfidf) #W matitx 
# get topcis
top_idx = np.argsort(nmf_embedding,axis=0)[-1:]
show_300 = 300
count_idxs = 0
final_names = data_words
topics_category_300 = []
for idxs in top_idx.T:
    if count_idxs == show_300: break
    for idx in idxs:
      temp = {}
      temp["topic_idx"] = count_idxs
      temp["topic"] = final_names[idx]
      topics_category_300.append(temp)
    count_idxs += 1
df_tpc300 = pd.DataFrame(topics_category_300)
nmf_training_set_topic = get_top_cat_for_recipe(df_title_val["title_clean"], nmf_embedding)

# testing
data_samples_test = [' '.join(item) for item in data_words_test]
tfidf_vectorizer =  pickle.load(open("vectorizer_nmf_07262021.pickle", 'rb'))     # Load vectorizer
tfidf_test = tfidf_vectorizer.transform(data_samples_test)
X_new = nmf_300.transform(tfidf_test) # test_embedding
categories_for_recipes_test = get_top_cat_for_recipe(data_words_test, X_new)
nmf_testing_set_topic = categories_for_recipes_test 

violation: 1.0
violation: 0.09097869529785488
violation: 0.04006592010450302
violation: 0.02146293049350556
violation: 0.011829907184339659
violation: 1.0
violation: 0.02770199537023632
violation: 0.0033312607564281436
violation: 0.0008261872022965138
violation: 0.00023664700648482907
violation: 1.0
violation: 0.027691286823504472
violation: 0.0033336732152642583
violation: 0.0008225169085590705
violation: 0.0002272286229144311


In [5]:
nmf_testing_set_topic = categories_for_recipes_test 

## Get the combined dataframe including id, title and corresponding topics for training/testing dataset

In [6]:
# Just run this function to get combined dataframe
def get_combined_dataframe(df_combined, categories_for_recipes):
  topic_list = []
  topic_idx_list = []
  for i in range(len(categories_for_recipes)):
    info = categories_for_recipes[i]
    if info == "no_cat":
      topic_list.append("No Cat")
      topic_idx_list.append("No Cat")
      continue
    topic_list.append(info["topic"])
    topic_idx_list.append(info["topic_idx"])
  df_combined["topic"] = topic_list
  df_combined["topic_idx"] = topic_idx_list
  return df_combined

In [7]:
# get combined dataframe including id, title, topic, topic index for each row
df_training_combined = get_combined_dataframe(training_df, nmf_training_set_topic)
df_testing_combined = get_combined_dataframe(testing_df, nmf_testing_set_topic)

## Traverse recipe id in df, find cooresponding image - create a dict {img_id : recipe id} & dataframe including all info

In [8]:
# input: recipe id, output: images.jpg(url could be printed here if needed)
def recipeid_to_image(recipe_id, print_url = False):
    images = []
    for food in layer2:
        if food.get("id") == recipe_id:
            image_list = food.get("images")
            for image in image_list:
                image_id = image.get("id")
                image_url = image.get("url")
                if print_url:
                    print(image_url) # this could be used for debug
                images.append(image_id)
            break
    return images

# input: recipe id, output: recipe details 
#                          (target_title, target_partition, target_url, target_ingredients, target_instructions)
def recipeid_to_details(recipe_id):
    target_ingredients = None
    target_url = None
    target_partition = None
    target_title = None
    target_instructions = None

    for recipe in layer1:
        if recipe.get("id") == recipe_id:
            target_ingredients = recipe.get("ingredients")
            target_url = recipe.get("url") 
            target_partition = recipe.get("partition")
            target_title = recipe.get("title") 
            target_instructions = recipe.get("instructions") 
            break
    return (target_title, target_partition, target_url, target_ingredients, target_instructions)
## layer_2 dict
def create_layer2_dict():
  layer2_dict = {}
  for food in layer2:
    recipe_id = food.get("id")
    image_list = food.get("images")
    image_names = []
    for image in image_list:
      image_names.append(image["id"])
    layer2_dict[recipe_id] = image_names
  return layer2_dict

def create_image_recipe_dict(this_df, layer2_dict):
    img_recipe_dict = {}
    for i in range(len(this_df)):
        # print(i)
        recipe_id = this_df.iloc[i]["id"]
        recipe_title = this_df.iloc[i]["title_clean"]
        recipe_topic = this_df.iloc[i]["topic"]
        recipe_topic_idx = this_df.iloc[i]["topic_idx"]
        # print(recipe_id)
        image_list = layer2_dict.get(recipe_id)
        # print(image_list)
        if image_list is None:
          continue
        for image in image_list:
            img_recipe_dict[image] = (recipe_id,recipe_title,recipe_topic,recipe_topic_idx )
        ## =========== remove this line or modify the number of recipes ==================
    return img_recipe_dict

def get_dataframe_all_info(df_all_info, dict_all):
  index = 0
  for key in dict_all:
    df_all_info.iloc[index]["Image name"] = key
    value = dict_all.get(key)
    df_all_info.iloc[index]["Recipe ID"] = value[0]
    df_all_info.iloc[index]["Title"] = value[1]
    df_all_info.iloc[index]["Topic"] = value[2]
    df_all_info.iloc[index]["Topic Index"] = value[3]
    index += 1
  return df_all_info

In [26]:
# get the dataframe with all info, remove "No cat", no need to change anything here, just run it
layer2_dict = create_layer2_dict()
dict_train = create_image_recipe_dict(df_training_combined, layer2_dict)
df_all_info_train = pd.DataFrame(index = range(0, len(dict_train)), columns=['Image name','Recipe ID','Title','Topic','Topic Index'])
df_all_info_train = get_dataframe_all_info(df_all_info_train, dict_train)
df_all_info_train = df_all_info_train[df_all_info_train.Topic != "No Cat"]

dict_test = create_image_recipe_dict(df_testing_combined, layer2_dict)
df_all_info_test = pd.DataFrame(index = range(0, len(dict_test)), columns=['Image name','Recipe ID','Title','Topic','Topic Index'])
df_all_info_test = get_dataframe_all_info(df_all_info_test, dict_test)
df_all_info_test = df_all_info_test[df_all_info_test.Topic != "No Cat"]

In [28]:
# shrinking size (don't run this cell if you are running a full training)
saved_info = df_all_info_train.copy()
shrinked_data_size = 1000
df_all_info_train = df_all_info_train[0:shrinked_data_size]
df_all_info_test = df_all_info_test[0:shrinked_data_size]

In [29]:
df_all_info_train

,Image name,Recipe ID,Title,Topic,Topic Index
0,665bbeafc7.jpg,00029f71f7,apple carrot bones dog treat,"[carrot, salad]",98
1,87aea5035b.jpg,00029f71f7,apple carrot bones dog treat,"[carrot, salad]",98
2,a97924d704.jpg,0004c091a0,whole wheat waffles,"[whole, wheat]",122
3,8e98aee7e1.jpg,000507ca6b,frosty strawberry squares,"[cheese, squares]",188
4,7027a5c4f9.jpg,0006354bc3,irresistible peanut butter cookies,"[peanut, butter]",10
...,...,...,...,...,...
505,e8cec314fa.jpg,01103b5d1b,cocktail grilled cheese appetizers,[cocktail],273
506,3352c4126b.jpg,0110c9c88d,grilled honey lemon chicken,"[lemon, lemon, chicken]",22
507,375e1cb026.jpg,0110c9c88d,grilled honey lemon chicken,"[lemon, lemon, chicken]",22
508,5178c517f6.jpg,0110c9c88d,grilled honey lemon chicken,"[lemon, lemon, chicken]",22


In [30]:
df_all_info_test

,Image name,Recipe ID,Title,Topic,Topic Index
0,3e233001e2.jpg,00003a70b1,crunchy onion potato bake,"[potato, salad]",11
1,7f749987f9.jpg,00003a70b1,crunchy onion potato bake,"[potato, salad]",11
2,aaf6b2dcd3.jpg,00003a70b1,crunchy onion potato bake,"[potato, salad]",11
3,1657f23729.jpg,00047059be,butternut squash soup or bisque roasting method,"[butternut, squash, soup]",68
4,7cd4a0f1a1.jpg,00047059be,butternut squash soup or bisque roasting method,"[butternut, squash, soup]",68
...,...,...,...,...,...
501,6e11dc989e.jpg,015dabf338,baked pumpkin rice pudding,"[pumpkin, salad]",28
502,a3beac4cf6.jpg,015dabf338,baked pumpkin rice pudding,"[pumpkin, salad]",28
503,de41580774.jpg,016067d95e,green ceviche with cucumber,[green],36
504,6b465ab497.jpg,01611a82ea,easy tiramisu cup,"[easy, easy, fruit, salad]",14


## Image loading

In [ ]:
# add image_name to topic index dict so that searching by image_name is easier
image_index_dict_train = {}

for i in range(len(df_all_info_train)):
  image_name = df_all_info_train.iloc[i]["Image name"]
  topic_idx = df_all_info_train.iloc[i]["Topic Index"]
  image_index_dict_train[image_name] = topic_idx 

image_index_dict_test = {}
for i in range(len(df_all_info_test)):
  image_name = df_all_info_test.iloc[i]["Image name"]
  topic_idx = df_all_info_test.iloc[i]["Topic Index"]
  image_index_dict_test[image_name] = topic_idx 

In [13]:
####### IMPORTANT LIST FOR TRAINING ##########
img_array_list = []
label_list = []
img_dir = "/content/gdrive/My Drive/2021 Summer CS7643 Share Folder/data/images/val"

############################################
def read_image(image_dir, target_img_name):
  first_dir = target_img_name[0]
  second_dir = target_img_name[1]
  third_dir = target_img_name[2]
  forth_dir = target_img_name[3]
  final_img_dir = image_dir + "/" + first_dir + "/" + second_dir + "/" + third_dir + "/" + forth_dir + "/" + target_img_name
  img = plt.imread(final_img_dir)
  return img


df_all_info_with_cat = df_all_info_train
# traverse all row in df_mini_batch and append the image array & title into list
for i in range(df_all_info_with_cat.shape[0]):
  selected_row = df_all_info_with_cat.iloc[i] # select ith row in df_mini_batch
  selected_label = selected_row["Topic Index"]
  selected_img_name = selected_row["Image name"]
  img_array = read_image(img_dir, selected_img_name) # return a (h,w,channel) np array
  # append into list
  img_array_list.append(img_array)
  label_list.append(selected_label)

# Create data for training from the list
my_x = np.zeros((len(img_array_list), 299, 299, 3)) # Inception accepts (299,299,3) image
for i in range(len(img_array_list)):
  img_new = cv2.resize(img_array_list[i], dsize=(299, 299), interpolation=cv2.INTER_CUBIC)
  my_x[i] = img_new
# Create label for training from the list
label = np.zeros((df_all_info_with_cat.shape[0], 1))
i = 0
for index, row in df_all_info_with_cat.iterrows():
    label[i] = df_all_info_with_cat["Topic Index"][index]
    #print(i,label[i])
    i += 1
label = label.astype(int)
label_encoded = F.one_hot(torch.tensor(label), num_classes=300)


In [31]:
####### IMPORTANT LIST FOR TESTING ##########
img_array_list_test = []
label_list_test = []
img_dir_test = "/content/gdrive/My Drive/2021 Summer CS7643 Share Folder/data/images/test"

############################################

# traverse all row in df_mini_batch and append the image array & title into list
for i in range(df_all_info_test.shape[0]):
  selected_row = df_all_info_test.iloc[i] # select ith row in df_mini_batch
  selected_label = selected_row["Topic Index"]
  selected_img_name = selected_row["Image name"]
  img_array = read_image(img_dir_test, selected_img_name) # return a (h,w,channel) np array
  # append into list
  img_array_list_test.append(img_array)
  label_list_test.append(selected_label)

# Create data for training from the list
my_x_test = np.zeros((len(img_array_list_test), 299, 299, 3)) # Inception accepts (299,299,3) image
for i in range(len(img_array_list_test)):
  img_new = cv2.resize(img_array_list_test[i], dsize=(299, 299), interpolation=cv2.INTER_CUBIC)
  my_x_test[i] = img_new
# Create label for training from the list
label_test = np.zeros((df_all_info_test.shape[0], 1))
i = 0
for index, row in df_all_info_test.iterrows():
    label_test[i] = df_all_info_test["Topic Index"][index]
    #print(i,label[i])
    i += 1
label_test = label_test.astype(int)

In [35]:
training_images = my_x
training_label = label
val_images = my_x_test
val_label = label_test

# Model building
# https://stackoverflow.com/questions/65594383/how-to-convert-dataset-of-images-to-tensor/65594790
# Self define a class which iterates over the dataset
class im2recipedataset(torch.utils.data.Dataset):
    def __init__(self, img_array, label_array):
      # load the dataset
      self.dataset = img_array
      self.label = label_array
  
    def __getitem__(self, idx):
        data = self.dataset[idx]
        label = self.label[idx]
                
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        return transform(data), label

    def __len__(self):
        return len(self.dataset)

train_set = im2recipedataset(training_images,training_label)
valid_set = im2recipedataset(val_images,val_label)

Data, label0 = train_set.__getitem__(0)

train_dataloader = DataLoader(train_set, batch_size=64, shuffle=True)
valid_dataloader = DataLoader(valid_set, batch_size=64, shuffle=True)
dataloaders_dict = {"train":train_dataloader, "val": valid_dataloader}

In [36]:
# ref: https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def initialize_model(num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    model_ft = models.inception_v3(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    # Handle the auxilary net
    num_ftrs = model_ft.AuxLogits.fc.in_features
    model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
    # Handle the primary net
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs,num_classes)
    input_size = 299

    return model_ft, input_size


#  Flag for feature extracting. When False, we finetune the whole model,
#  when True we only update the reshaped layer params
feature_extract = True

# Initialize the model for this run
InceptionV3, input_size = initialize_model(300, feature_extract, use_pretrained=True)

# Print the model we just instantiated
#print(InceptionV3)

# https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
# Send the model to GPU
InceptionV3 = InceptionV3.to(device)

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth



cuda:0


In [37]:
# parameter for model hyperparameter tuning
learning_rate = 0.001
momentum_value = 0.9
criterion_type = nn.CrossEntropyLoss()
num_epochs = 200

#=============================================

params_to_update = InceptionV3.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in InceptionV3.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=learning_rate , momentum=momentum_value)
# Setup the loss fxn
criterion = criterion_type

Params to learn:
	 AuxLogits.fc.weight
	 AuxLogits.fc.bias
	 fc.weight
	 fc.bias


In [38]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                inputs = inputs.type(torch.cuda.FloatTensor)
                #print(inputs.shape)
                labels = torch.squeeze(labels)
                labels = labels.to(device)
                #print(labels.shape)
                #outputs, aux_outputs = model(inputs)
                #print(outputs.shape)
                #print(aux_outputs.shape)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [39]:
# Train and evaluate
model_ft, hist = train_model(InceptionV3, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=True)

Epoch 0/199
----------


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


train Loss: 8.0484 Acc: 0.0000
val Loss: 5.7177 Acc: 0.0040

Epoch 1/199
----------
train Loss: 7.9344 Acc: 0.0100
val Loss: 5.7079 Acc: 0.0080

Epoch 2/199
----------
train Loss: 7.7502 Acc: 0.0540
val Loss: 5.6986 Acc: 0.0060

Epoch 3/199
----------
train Loss: 7.5835 Acc: 0.0720
val Loss: 5.7031 Acc: 0.0040

Epoch 4/199
----------
train Loss: 7.3715 Acc: 0.0700
val Loss: 5.7198 Acc: 0.0060

Epoch 5/199
----------
train Loss: 7.2264 Acc: 0.0760
val Loss: 5.7440 Acc: 0.0040

Epoch 6/199
----------
train Loss: 7.1015 Acc: 0.0780
val Loss: 5.7686 Acc: 0.0060

Epoch 7/199
----------
train Loss: 6.9862 Acc: 0.0760
val Loss: 5.7898 Acc: 0.0080

Epoch 8/199
----------
train Loss: 6.8728 Acc: 0.0860
val Loss: 5.8056 Acc: 0.0080

Epoch 9/199
----------
train Loss: 6.7803 Acc: 0.0880
val Loss: 5.8228 Acc: 0.0080

Epoch 10/199
----------
train Loss: 6.7027 Acc: 0.1040
val Loss: 5.8374 Acc: 0.0080

Epoch 11/199
----------
train Loss: 6.6129 Acc: 0.1100
val Loss: 5.8535 Acc: 0.0080

Epoch 12/199


In [ ]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
print(input.shape)
target = torch.empty(3, dtype=torch.long).random_(5)
print(target.shape)
#output = loss(input, target)
#output.backward()

torch.Size([3, 5])
torch.Size([3])
